Connected to quant-stack (Python 3.11.9)

In [ ]:
import os
from pathlib import Path

# Data Analysis
import pandas as pd
# import pytimetk as tk
import polars as pl

# Files

import yaml


# QSConnect 
from qsconnect import Client


# * ENVIRONMENT VARIABLES ----

# Constants
TODAY = "2025-07-28"

In [ ]:
from pathlib import Path

In [ ]:
import pandas as pd
# import pytimetk as tk
import polars as pl

# Files

import yaml

In [ ]:
import polars as pl

In [ ]:
import yaml

In [ ]:
Client

qsconnect.client.Client

In [ ]:
TODAY = "2025-07-28"

In [ ]:
os.environ['FMP_API_KEY'] = yaml.safe_load(open('credentials.yml', encoding='utf-8'))['fmp_api']

KeyError: 'fmp_api'

In [ ]:
os.environ['FMP_API_KEY'] = yaml.safe_load(open('credentials.yml', encoding='utf-8'))['fmp_api_key']

In [ ]:
os.environ['FMP_API_KEY']

'EGUXlkKgIAY9jvTbGDXWb1WY4CAVfjgE'

In [ ]:
# QSCONNECT PATH TO CACHE & DATABASES
os.environ['QSCONNECT_ROOT'] = str(Path.cwd() / 'data' / 'qsconnect')

os.environ['QSCONNECT_ROOT'] = str(Path.cwd() / 'data' / 'qsconnect-demo')

In [ ]:
client = Client()

In [ ]:
client

QSConnect(fmp_api_key=***WY4CAVfjgE, datalink_api_key=None)

In [ ]:
client.cache_dir

PosixPath('/Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/data/qsconnect-demo/cache')

In [ ]:
client.user_cache_dir

PosixPath('/Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/data/qsconnect-demo/cache/user')

In [ ]:
client.fmp_cache_dir

PosixPath('/Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/data/qsconnect-demo/cache/fmp')

In [ ]:
client.yahoo_cache_dir

PosixPath('/Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/data/qsconnect-demo/cache/yahoo')

In [ ]:
client.database_dir

PosixPath('/Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/data/qsconnect-demo/database')

In [ ]:
stock_list = client.stock_list(
    security_type="stock", 
    override_date=TODAY
)

2025-07-28 18:03:21,462 - QSConnect - WARNING - No cache file found for key 'stock_list-2025-07-28' at /Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/data/qsconnect-demo/cache/fmp/stock_list-2025-07-28.parquet
INFO [2025-07-28T18:03:25-0400][QSLog] Parsing response with content length: 16200978 bytes
INFO [2025-07-28T18:03:25-0400][QSLog] Successfully parsed response into a Pandas DataFrame.
2025-07-28 18:03:25,672 - QSConnect - INFO - Caching data at /Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/data/qsconnect-demo/cache/fmp
2025-07-28 18:03:25,715 - QSConnect - INFO - Cached data for key 'stock_list-2025-07-28' to /Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/data/qsconnect-demo/cache/fmp/stock_list-2025-07-28.parquet


In [ ]:
stock_list

,symbol,name,price,exchange,exchangeShortName,type
0,KZMS.ME,The Open Joint Stock Company Krasnokamsk Metal...,226.0000,Moscow Stock Exchange,MCX,stock
1,QREARX,TIAA Real Estate Account,470.6850,NASDAQ,NASDAQ,trust
2,FEMACX,The First Trust Combined Series 447: Investmen...,911.2000,NASDAQ,NASDAQ,trust
3,QCEQPX,College Retirement Equities Fund - CREF Equity...,559.1039,NASDAQ,NASDAQ,trust
4,QCGRIX,College Retirement Equities Fund - CREF Growth...,583.6861,NASDAQ,NASDAQ,trust
...,...,...,...,...,...,...
85743,TENKR,TenX Keane Acquisition Right,0.4800,NASDAQ,NASDAQ,stock
85744,VHNAW,Vahanna Tech Edge Acquisition I Corp.,0.1229,NASDAQ Global Market,NASDAQ,stock
85745,AFTR-WT,AfterNext HealthTech Acquisition Corp.,0.0041,New York Stock Exchange,NYSE,stock
85746,STA.L,Star Phoenix Group Ltd,1.0500,London Stock Exchange,LSE,stock


In [ ]:
stock_list.glimpse()

AttributeError: 'DataFrame' object has no attribute 'glimpse'

In [ ]:
stock_list

stock_list.info()

stock_list.glimpse()

stock_list[stock_list['symbol'] == 'NVDA']

# Asset Types

stock_list['type'].value_counts()


# Exchange Types

stock_list[['exchange', 'exchangeShortName']].value_counts()


# Create a list of symbols to get prices for

asset_types = ['stock'] 
exchange_types = ['NASDAQ', 'NYSE', 'AMEX']

symbols_to_get = []

for asset_type in asset_types:
    for exchange_type in exchange_types:
        filtered_symbols = stock_list[
            (stock_list['type'] == asset_type) & 
            (stock_list['exchangeShortName'] == exchange_type)
        ]['symbol'].tolist()
        symbols_to_get.extend(filtered_symbols)

symbols_to_get[0:10]  # Display the first 10 symbols

len(symbols_to_get)  # Total number of symbols to get prices for

# Add SPY and QQQ to the list for Benchmarking
symbols_to_get = ['SPY', 'QQQ'] + symbols_to_get


# * Step 2: Get the stock prices for all tickers

# * FMP Prices API

# FMP API Notes: 
#  1. This will take 10 minutes to run as it fetches historical prices for 12,000+ assets and convert to a 3 GB parquet file.
#  2. You can adjust the `api_calls_per_minute` parameter based on your FMP plan to avoid hitting rate limits. My limit is 3000 calls per minute, so I set it to 2000 here because I saw I was getting ERRORS when I set it to 3000.

prices_path = client.historical_prices(
    symbols=symbols_to_get[:2000],
    start_date="2000-01-01",
    end_date=TODAY,
    api_calls_per_minute=2000, # Adjust based on your FMP plan
    cache=True,  # This will cache the file in the FMP cache directory
    return_path=True,  # This will return the path to the cached file instead of the DataFrame (which is useful for large datasets)
)

# Inspect the cached file using Polars Lazy Evaluation for large datasets

# * Your cached file path will be different, so you need to adjust this path accordingly
prices_path = '/Users/mdancho/Desktop/course_code/QS02-Quant_Scientist_Algo_Trading_System/data/qsconnect/cache/fmp/historical-prices-2025-07-17.parquet'

# Detect Missing Symbols

# Get the head of the DataFrame to see the first few rows
lazy_df = pl.scan_parquet(prices_path)
lazy_df.head().collect().to_pandas()

# Get the unique symbols in the DataFrame
lazy_df = (
    pl.scan_parquet(prices_path)
    .select(pl.col("symbol").unique())
)   

unique_symbols = lazy_df.collect().to_pandas()

unique_symbols

# Difference between symbols_to_get and unique_symbols
missing_symbols = list(set(symbols_to_get) - set(unique_symbols['symbol'].tolist()))

pd.DataFrame(missing_symbols, columns=['symbol'])


# Locate a single stock ticker, e.g., Apple (AAPL)

lazy_df = (
    pl.scan_parquet(prices_path)
    .filter(pl.col("symbol") == "AAPL")
)

df = lazy_df.collect().to_pandas()

df

df.glimpse()

df \
    .groupby('symbol') \
    .plot_timeseries(
        date_column='date',
        value_column='close',
        title='Stock Price',
    )


# * Yahoo Finance API Downloads

# Note: This is a long running operation (about 10 minutes)

client.historical_prices_yahoo(
    symbols=symbols_to_get[:2000],  
    start_date="2000-01-01",
    end_date=TODAY,
    batch_size=100,
)


# * Step 3: Get the fundamental data for all tickers

# - WARNING: Long running operation (134 minutes)
# - Note: Must wait 10 seconds between bulk API requests

client.fetch_bulk_financial_statements(
    statement_type='all', # all available statements: income, balance, cash flow, ...
    periods='all', # annual and quarter 
    
    # start_year=2000,
    start_year=2024,  
    
    end_year=pd.to_datetime(TODAY).year,
    api_buffer_seconds=12,  # Adjust based on your FMP plan to avoid hitting rate limits
)

# * Step 4: Detect Cached Files

client.detect_cached_files()

client.summarize_cached_files()


# Detect Missing Cached Files

df = client.detect_missing_cached_files(
    statement_type="all",
    periods='all',
    start_year=2001,
    end_year=2025, # Try with 2025
    override_date=TODAY,
)

# Clean up Cached Files
help(client.delete_cached_files)

# client.delete_cached_files(df.iloc[[0]])

# * 2.0 BUILD YOUR QUANTAMENTAL FINANCIAL DATABASE ----


# * Step 1: Identify the files you want to include in your database

all_cached_files = client.detect_cached_files(
    # start_date=TODAY,
    # end_date=pd.to_datetime(TODAY) + pd.DateOffset(days=1),  
)

all_cached_files[all_cached_files['file_name'].str.startswith('historical-prices')]

all_cached_files[all_cached_files['file_name'].str.contains('2006')]

# * Step 2: Create a database from the cached files

# This takes a 5 minutes to run as it processes all the cached files and creates the qsconnect DuckDB database

errors_df = client.load_cached_files_to_database(
    cached_files_df=all_cached_files, 
    fresh=True, # Set to True to create a fresh database; set to False to append to an existing database
)

errors_df # If this is empty, then all files were loaded successfully; otherwise a DataFrame with errors will be returned

# * Step 3: Inspect the database

# Connect to the database
client.connect_to_database()

# List All Database Tables
client.list_database_tables()

# Glimpse All Database Tables
client.glimpse_all_database_tables(max_colwidth=25)

# Missing Values Report
client.missing_values_all_database_tables(max_colwidth=25)

# Missing Calendar Years Report for FMP Fundamental Data
client.missing_calendar_years_all_database_tables(
    start_year=2000, 
    end_year=2025,
)

# Quality Report
client.database_quality_report(
    start_year=2000, 
    end_year=2025
)

# Run SQL Queries on the Database
conn = client.connect_to_database()

conn.execute("""
    SELECT *
    FROM historical_prices_fmp
    LIMIT 5
""").fetchdf()

# Disconnect from the database
client.close_database_connection()

# DBCode Extension Demo
# - Use the DBCode extension to examine the database tables


# * 3.0 BUILD YOUR ZIPLINE BUNDLE ----

# * Step 1: Create a Zipline Bundle from the FMP Database

# Connect to the database
client.connect_to_database()

# Create a Zipline Bundle from the database
client.ingest_zipline_bundle_from_fmp_tables(bundle_name="qspro_demo_historical_prices_fmp")

# Disconnect from the database
client.close_database_connection()

# * Step 2: Open the Zipline hidden folder

# Mac users can find the hidden folder at:
#   1. Open Finder.
#   2. Press `Command + Shift + .` (period) to toggle hidden files visibility.
#   3. Navigate to the `~/.zipline` directory.

# Windows users can find the hidden folder at:
#   C:\Users\<YourUsername>\.zipline

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85748 entries, 0 to 85747
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symbol             85748 non-null  object 
 1   name               85748 non-null  object 
 2   price              85708 non-null  float64
 3   exchange           85680 non-null  object 
 4   exchangeShortName  85687 non-null  object 
 5   type               85748 non-null  object 
dtypes: float64(1), object(5)
memory usage: 3.9+ MB


AttributeError: 'DataFrame' object has no attribute 'glimpse'

In [ ]:
stock_list.shape()

TypeError: 'tuple' object is not callable

In [ ]:
stock_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85748 entries, 0 to 85747
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symbol             85748 non-null  object 
 1   name               85748 non-null  object 
 2   price              85708 non-null  float64
 3   exchange           85680 non-null  object 
 4   exchangeShortName  85687 non-null  object 
 5   type               85748 non-null  object 
dtypes: float64(1), object(5)
memory usage: 3.9+ MB


In [ ]:
stock_list.shape

(85748, 6)

In [ ]:
stock_list[stock_list['symbol'] == 'NVDA']

,symbol,name,price,exchange,exchangeShortName,type
9151,NVDA,NVIDIA Corporation,176.75,NASDAQ Global Select,NASDAQ,stock


In [ ]:
stock_list['type'].value_counts()

type
stock    66137
etf      13139
trust     4266
fund      2206
Name: count, dtype: int64

In [ ]:
stock_list[['exchange', 'exchangeShortName']].value_counts()

exchange                    exchangeShortName
Other OTC                   OTC                  13949
London Stock Exchange       LSE                   6805
NASDAQ                      NASDAQ                5322
New York Stock Exchange     NYSE                  4465
Bombay Stock Exchange       BSE                   4136
                                                 ...  
Stock Exchange of Thailand  SES                      1
Colombia Stock Exchange     BVC                      1
Ho Chi Minh Stock Exchange  HOSE                     1
NASDAQ Riga                 RIS                      1
NASDAQ Global Market        CBOE                     1
Name: count, Length: 79, dtype: int64

In [ ]:
asset_types = ['stock'] 
exchange_types = ['NASDAQ', 'NYSE', 'AMEX']

symbols_to_get = []

for asset_type in asset_types:
    for exchange_type in exchange_types:
        filtered_symbols = stock_list[
            (stock_list['type'] == asset_type) & 
            (stock_list['exchangeShortName'] == exchange_type)
        ]['symbol'].tolist()
        symbols_to_get.extend(filtered_symbols)

symbols_to_get[0:10]  # Display the first 10 symbols

len(symbols_to_get)  # Total number of symbols to get prices for

# Add SPY and QQQ to the list for Benchmarking
symbols_to_get = ['SPY', 'QQQ'] + symbols_to_get

In [ ]:
asset_types = ['stock']

In [ ]:
exchange_types = ['NASDAQ', 'NYSE', 'AMEX']

In [ ]:
for asset_type in asset_types:
    for exchange_type in exchange_types:
        filtered_symbols = stock_list[
            (stock_list['type'] == asset_type) & 
            (stock_list['exchangeShortName'] == exchange_type)
        ]['symbol'].tolist()
        symbols_to_get.extend(filtered_symbols)

In [ ]:
symbols_to_get[0:10]

['SPY', 'QQQ', 'ICU', 'KALV', 'MBUU', 'ALGN', 'HBNC', 'EVER', 'CBAN', 'ANGH']

In [ ]:
len(symbols_to_get)

23270

In [ ]:
symbols_to_get = ['SPY', 'QQQ'] + symbols_to_get

In [ ]:
asset_types = ['stock'] 
exchange_types = ['NASDAQ', 'NYSE', 'AMEX']

symbols_to_get = []

for asset_type in asset_types:
    for exchange_type in exchange_types:
        filtered_symbols = stock_list[
            (stock_list['type'] == asset_type) & 
            (stock_list['exchangeShortName'] == exchange_type)
        ]['symbol'].tolist()
        symbols_to_get.extend(filtered_symbols)

symbols_to_get[0:10]  # Display the first 10 symbols

len(symbols_to_get)  # Total number of symbols to get prices for

# Add SPY and QQQ to the list for Benchmarking
symbols_to_get = ['SPY', 'QQQ'] + symbols_to_get

symbols_to_get

['SPY',
 'QQQ',
 'ICU',
 'KALV',
 'MBUU',
 'ALGN',
 'HBNC',
 'EVER',
 'CBAN',
 'ANGH',
 'ADTX',
 'FFIV',
 'MSEX',
 'ASRV',
 'CRWV',
 'IVF',
 'LFVN',
 'CENN',
 'DOMH',
 'ACTU',
 'GOCO',
 'MRUS',
 'HBT',
 'HNVR',
 'API',
 'AOUT',
 'BKR',
 'BRZE',
 'DAVE',
 'DTSS',
 'LBTYK',
 'ABVC',
 'BANF',
 'DFSC',
 'DDOG',
 'GOGL',
 'HOOD',
 'STEL',
 'KINS',
 'ENSC',
 'ENSG',
 'LIVN',
 'EWBC',
 'EXEL',
 'APEI',
 'FLXS',
 'MLEC',
 'HAFC',
 'HXHX',
 'IBKR',
 'MPWR',
 'VIRT',
 'INKT',
 'CAMT',
 'BIYA',
 'AKRO',
 'CLAR',
 'HBCP',
 'LINC',
 'AFYA',
 'APA',
 'ARTNA',
 'ATHE',
 'COST',
 'MDAI',
 'ACHV',
 'ANTX',
 'COKE',
 'APVO',
 'BCAX',
 'CWBC',
 'BETR',
 'ADPT',
 'AVAH',
 'FFIC',
 'ILMN',
 'MSAI',
 'CORZ',
 'ASML',
 'BPOP',
 'KROS',
 'GNTX',
 'BGLC',
 'KRRO',
 'MMYT',
 'AOSL',
 'JOUT',
 'CGBD',
 'IMTE',
 'HOLX',
 'QXO',
 'AMD',
 'BLIN',
 'ALAR',
 'ARQT',
 'ASTS',
 'MBCN',
 'LYFT',
 'FSV',
 'GRAL',
 'ARHS',
 'GWAV',
 'PRIM',
 'BFC',
 'BLFS',
 'FAST',
 'KE',
 'HOWL',
 'ANGO',
 'CME',
 'LQDA',
 'MRX',
 'ESOA

In [ ]:
prices_path = client.historical_prices(
    symbols=symbols_to_get[:2000],
    start_date="2000-01-01",
    end_date=TODAY,
    api_calls_per_minute=2000, # Adjust based on your FMP plan
    cache=True,  # This will cache the file in the FMP cache directory
    return_path=True,  # This will return the path to the cached file instead of the DataFrame (which is useful for large datasets)
)

2025-07-28 18:27:04,034 - QSConnect - INFO - API calls limited to 2000 per 60 seconds.
Fetching data: 100%|██████████| 2000/2000 [00:25<00:00, 79.57symbol/s] 
2025-07-28 18:27:29,225 - QSConnect - INFO - Data fetching completed in 25.19 seconds


In [ ]:
prices_path = '/Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/data/qsconnect/cache/fmp/historical-prices-2025-07-28.parquet'

In [ ]:
lazy_df = pl.scan_parquet(prices_path)

In [ ]:
lazy_df.head().collect().to_pandas()

FileNotFoundError: No such file or directory (os error 2): ...ident/QS-Project/Clinic-05/data/qsconnect/cache/fmp/historical-prices-2025-07-28.parquet (set POLARS_VERBOSE=1 to see full path)

This error occurred with the following context stack:
	[1] 'parquet scan'
	[2] 'slice'
	[3] 'sink'


In [ ]:
# * Step 2: Get the stock prices for all tickers

# * FMP Prices API

# FMP API Notes: 
#  1. This will take 10 minutes to run as it fetches historical prices for 12,000+ assets and convert to a 3 GB parquet file.
#  2. You can adjust the `api_calls_per_minute` parameter based on your FMP plan to avoid hitting rate limits. My limit is 3000 calls per minute, so I set it to 2000 here because I saw I was getting ERRORS when I set it to 3000.

prices_path = client.historical_prices(
    symbols=symbols_to_get[:2000],
    start_date="2000-01-01",
    end_date=TODAY,
    api_calls_per_minute=2000, # Adjust based on your FMP plan
    cache=True,  # This will cache the file in the FMP cache directory
    return_path=True,  # This will return the path to the cached file instead of the DataFrame (which is useful for large datasets)
)

# Inspect the cached file using Polars Lazy Evaluation for large datasets

# * Your cached file path will be different, so you need to adjust this path accordingly
# prices_path = '/Users/mdancho/Desktop/course_code/QS02-Quant_Scientist_Algo_Trading_System/data/qsconnect/cache/fmp/historical-prices-2025-07-17.parquet'
prices_path = '/Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/data/qsconnect/cache/fmp/historical-prices-2025-07-28.parquet'
# Detect Missing Symbols

# Get the head of the DataFrame to see the first few rows
lazy_df = pl.scan_parquet(prices_path)

lazy_df.info()

lazy_df.head().collect().to_pandas()

# Get the unique symbols in the DataFrame
lazy_df = (
    pl.scan_parquet(prices_path)
    .select(pl.col("symbol").unique())
)   

unique_symbols = lazy_df.collect().to_pandas()

unique_symbols

# Difference between symbols_to_get and unique_symbols
missing_symbols = list(set(symbols_to_get) - set(unique_symbols['symbol'].tolist()))

pd.DataFrame(missing_symbols, columns=['symbol'])


# Locate a single stock ticker, e.g., Apple (AAPL)

lazy_df = (
    pl.scan_parquet(prices_path)
    .filter(pl.col("symbol") == "AAPL")
)

df = lazy_df.collect().to_pandas()

df

df.glimpse()

df \
    .groupby('symbol') \
    .plot_timeseries(
        date_column='date',
        value_column='close',
        title='Stock Price',
    )


# * Yahoo Finance API Downloads

# Note: This is a long running operation (about 10 minutes)

client.historical_prices_yahoo(
    symbols=symbols_to_get[:2000],  
    start_date="2000-01-01",
    end_date=TODAY,
    batch_size=100,
)


# * Step 3: Get the fundamental data for all tickers

# - WARNING: Long running operation (134 minutes)
# - Note: Must wait 10 seconds between bulk API requests

client.fetch_bulk_financial_statements(
    statement_type='all', # all available statements: income, balance, cash flow, ...
    periods='all', # annual and quarter 
    
    # start_year=2000,
    start_year=2024,  
    
    end_year=pd.to_datetime(TODAY).year,
    api_buffer_seconds=12,  # Adjust based on your FMP plan to avoid hitting rate limits
)

# * Step 4: Detect Cached Files

client.detect_cached_files()

client.summarize_cached_files()


# Detect Missing Cached Files

df = client.detect_missing_cached_files(
    statement_type="all",
    periods='all',
    start_year=2001,
    end_year=2025, # Try with 2025
    override_date=TODAY,
)

# Clean up Cached Files
help(client.delete_cached_files)

# client.delete_cached_files(df.iloc[[0]])

# * 2.0 BUILD YOUR QUANTAMENTAL FINANCIAL DATABASE ----


# * Step 1: Identify the files you want to include in your database

all_cached_files = client.detect_cached_files(
    # start_date=TODAY,
    # end_date=pd.to_datetime(TODAY) + pd.DateOffset(days=1),  
)

all_cached_files[all_cached_files['file_name'].str.startswith('historical-prices')]

all_cached_files[all_cached_files['file_name'].str.contains('2006')]

# * Step 2: Create a database from the cached files

# This takes a 5 minutes to run as it processes all the cached files and creates the qsconnect DuckDB database

errors_df = client.load_cached_files_to_database(
    cached_files_df=all_cached_files, 
    fresh=True, # Set to True to create a fresh database; set to False to append to an existing database
)

errors_df # If this is empty, then all files were loaded successfully; otherwise a DataFrame with errors will be returned

# * Step 3: Inspect the database

# Connect to the database
client.connect_to_database()

# List All Database Tables
client.list_database_tables()

# Glimpse All Database Tables
client.glimpse_all_database_tables(max_colwidth=25)

# Missing Values Report
client.missing_values_all_database_tables(max_colwidth=25)

# Missing Calendar Years Report for FMP Fundamental Data
client.missing_calendar_years_all_database_tables(
    start_year=2000, 
    end_year=2025,
)

# Quality Report
client.database_quality_report(
    start_year=2000, 
    end_year=2025
)

# Run SQL Queries on the Database
conn = client.connect_to_database()

conn.execute("""
    SELECT *
    FROM historical_prices_fmp
    LIMIT 5
""").fetchdf()

# Disconnect from the database
client.close_database_connection()

# DBCode Extension Demo
# - Use the DBCode extension to examine the database tables


# * 3.0 BUILD YOUR ZIPLINE BUNDLE ----

# * Step 1: Create a Zipline Bundle from the FMP Database

# Connect to the database
client.connect_to_database()

# Create a Zipline Bundle from the database
client.ingest_zipline_bundle_from_fmp_tables(bundle_name="qspro_demo_historical_prices_fmp")

# Disconnect from the database
client.close_database_connection()

# * Step 2: Open the Zipline hidden folder

# Mac users can find the hidden folder at:
#   1. Open Finder.
#   2. Press `Command + Shift + .` (period) to toggle hidden files visibility.
#   3. Navigate to the `~/.zipline` directory.

# Windows users can find the hidden folder at:
#   C:\Users\<YourUsername>\.zipline

2025-07-28 18:46:39,400 - QSConnect - INFO - Loaded cached data for the given date range from cache.


AttributeError: 'LazyFrame' object has no attribute 'info'

In [ ]:
lazy_df.head

<bound method LazyFrame.head of <LazyFrame at 0x32A4F9B50>>

In [ ]:
lazy_df.head

<bound method LazyFrame.head of <LazyFrame at 0x32A4F9B50>>

In [ ]:
prices_path = '/Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/'
              'data/qsconnect-demo/cache/fmp/historical-prices-2025-07-28.parquet'

IndentationError: unexpected indent (<ipython-input-44-164d7fc20cfe>, line 2)

In [ ]:
prices_path = '/Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/'
'data/qsconnect-demo/cache/fmp/historical-prices-2025-07-28.parquet'

'data/qsconnect-demo/cache/fmp/historical-prices-2025-07-28.parquet'

In [ ]:
# * Step 2: Get the stock prices for all tickers

# * FMP Prices API

# FMP API Notes: 
#  1. This will take 10 minutes to run as it fetches historical prices for 12,000+ assets and convert to a 3 GB parquet file.
#  2. You can adjust the `api_calls_per_minute` parameter based on your FMP plan to avoid hitting rate limits. My limit is 3000 calls per minute, so I set it to 2000 here because I saw I was getting ERRORS when I set it to 3000.

prices_path = client.historical_prices(
    symbols=symbols_to_get[:2000],
    start_date="2000-01-01",
    end_date=TODAY,
    api_calls_per_minute=2000, # Adjust based on your FMP plan
    cache=True,  # This will cache the file in the FMP cache directory
    return_path=True,  # This will return the path to the cached file instead of the DataFrame (which is useful for large datasets)
)

# Inspect the cached file using Polars Lazy Evaluation for large datasets

# * Your cached file path will be different, so you need to adjust this path accordingly
# prices_path = '/Users/mdancho/Desktop/course_code/QS02-Quant_Scientist_Algo_Trading_System/data/qsconnect/cache/fmp/historical-prices-2025-07-17.parquet'
prices_path = '/Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/'
'data/qsconnect-demo/cache/fmp/historical-prices-2025-07-28.parquet'

print(prices_path)

# Detect Missing Symbols

# Get the head of the DataFrame to see the first few rows
lazy_df = pl.scan_parquet(prices_path)

lazy_df.head

lazy_df.head().collect().to_pandas()

# Get the unique symbols in the DataFrame
lazy_df = (
    pl.scan_parquet(prices_path)
    .select(pl.col("symbol").unique())
)   

unique_symbols = lazy_df.collect().to_pandas()

unique_symbols

# Difference between symbols_to_get and unique_symbols
missing_symbols = list(set(symbols_to_get) - set(unique_symbols['symbol'].tolist()))

pd.DataFrame(missing_symbols, columns=['symbol'])


# Locate a single stock ticker, e.g., Apple (AAPL)

lazy_df = (
    pl.scan_parquet(prices_path)
    .filter(pl.col("symbol") == "AAPL")
)

df = lazy_df.collect().to_pandas()

df

df.glimpse()

df \
    .groupby('symbol') \
    .plot_timeseries(
        date_column='date',
        value_column='close',
        title='Stock Price',
    )


# * Yahoo Finance API Downloads

# Note: This is a long running operation (about 10 minutes)

client.historical_prices_yahoo(
    symbols=symbols_to_get[:2000],  
    start_date="2000-01-01",
    end_date=TODAY,
    batch_size=100,
)


# * Step 3: Get the fundamental data for all tickers

# - WARNING: Long running operation (134 minutes)
# - Note: Must wait 10 seconds between bulk API requests

client.fetch_bulk_financial_statements(
    statement_type='all', # all available statements: income, balance, cash flow, ...
    periods='all', # annual and quarter 
    
    # start_year=2000,
    start_year=2024,  
    
    end_year=pd.to_datetime(TODAY).year,
    api_buffer_seconds=12,  # Adjust based on your FMP plan to avoid hitting rate limits
)

# * Step 4: Detect Cached Files

client.detect_cached_files()

client.summarize_cached_files()


# Detect Missing Cached Files

df = client.detect_missing_cached_files(
    statement_type="all",
    periods='all',
    start_year=2001,
    end_year=2025, # Try with 2025
    override_date=TODAY,
)

# Clean up Cached Files
help(client.delete_cached_files)

# client.delete_cached_files(df.iloc[[0]])

# * 2.0 BUILD YOUR QUANTAMENTAL FINANCIAL DATABASE ----


# * Step 1: Identify the files you want to include in your database

all_cached_files = client.detect_cached_files(
    # start_date=TODAY,
    # end_date=pd.to_datetime(TODAY) + pd.DateOffset(days=1),  
)

all_cached_files[all_cached_files['file_name'].str.startswith('historical-prices')]

all_cached_files[all_cached_files['file_name'].str.contains('2006')]

# * Step 2: Create a database from the cached files

# This takes a 5 minutes to run as it processes all the cached files and creates the qsconnect DuckDB database

errors_df = client.load_cached_files_to_database(
    cached_files_df=all_cached_files, 
    fresh=True, # Set to True to create a fresh database; set to False to append to an existing database
)

errors_df # If this is empty, then all files were loaded successfully; otherwise a DataFrame with errors will be returned

# * Step 3: Inspect the database

# Connect to the database
client.connect_to_database()

# List All Database Tables
client.list_database_tables()

# Glimpse All Database Tables
client.glimpse_all_database_tables(max_colwidth=25)

# Missing Values Report
client.missing_values_all_database_tables(max_colwidth=25)

# Missing Calendar Years Report for FMP Fundamental Data
client.missing_calendar_years_all_database_tables(
    start_year=2000, 
    end_year=2025,
)

# Quality Report
client.database_quality_report(
    start_year=2000, 
    end_year=2025
)

# Run SQL Queries on the Database
conn = client.connect_to_database()

conn.execute("""
    SELECT *
    FROM historical_prices_fmp
    LIMIT 5
""").fetchdf()

# Disconnect from the database
client.close_database_connection()

# DBCode Extension Demo
# - Use the DBCode extension to examine the database tables


# * 3.0 BUILD YOUR ZIPLINE BUNDLE ----

# * Step 1: Create a Zipline Bundle from the FMP Database

# Connect to the database
client.connect_to_database()

# Create a Zipline Bundle from the database
client.ingest_zipline_bundle_from_fmp_tables(bundle_name="qspro_demo_historical_prices_fmp")

# Disconnect from the database
client.close_database_connection()

# * Step 2: Open the Zipline hidden folder

# Mac users can find the hidden folder at:
#   1. Open Finder.
#   2. Press `Command + Shift + .` (period) to toggle hidden files visibility.
#   3. Navigate to the `~/.zipline` directory.

# Windows users can find the hidden folder at:
#   C:\Users\<YourUsername>\.zipline

2025-07-28 19:37:17,215 - QSConnect - INFO - Loaded cached data for the given date range from cache.


/Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/


InvalidOperationError: directory contained paths with different file extensions: \
                first path: /Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/.DS_Store, second path: /Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/01_quantamental_financial_database_setup.py. Please use a glob pattern to explicitly specify \
                which files to read (e.g. "dir/**/*", "dir/**/*.parquet")

This error occurred with the following context stack:
	[1] 'slice'
	[2] 'sink'


In [ ]:
print(prices_path)

/Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/


In [ ]:
'

SyntaxError: unterminated string literal (detected at line 1) (<ipython-input-48-82eddc4252ed>, line 1)

In [ ]:
prices_path =  "/Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/"
"data/qsconnect-demo/cache/fmp/historical-prices-2025-07-28.parquet"

'data/qsconnect-demo/cache/fmp/historical-prices-2025-07-28.parquet'

In [ ]:
print(prices_path)

/Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/


In [ ]:
prices_path =  "/Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/" + 
"data/qsconnect-demo/cache/fmp/historical-prices-2025-07-28.parquet"

SyntaxError: invalid syntax (<ipython-input-51-e5e76e9d0ea6>, line 1)

In [ ]:
prices_path =  "/Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/" + "data/qsconnect-demo/cache/fmp/historical-prices-2025-07-28.parquet"

In [ ]:
print(prices_path)

/Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/data/qsconnect-demo/cache/fmp/historical-prices-2025-07-28.parquet


In [ ]:
lazy_df = pl.scan_parquet(prices_path)

In [ ]:
lazy_df.head().collect().to_pandas()

,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,symbol
0,2025-07-28,1.82,1.88,1.56,1.64,1.64,1540007.0,1540007.0,-0.18,-9.89,1.72,"July 28, 25",-0.0989,HIT
1,2025-07-25,2.01,2.12,1.83,1.84,1.84,1239700.0,1239700.0,-0.17,-8.46,1.95,"July 25, 25",-0.0846,HIT
2,2025-07-24,1.92,2.00,1.76,1.94,1.94,1817800.0,1817800.0,0.02,1.04,1.91,"July 24, 25",0.0104,HIT
3,2025-07-23,1.79,2.02,1.73,1.81,1.81,3674500.0,3674500.0,0.02,1.12,1.84,"July 23, 25",0.0112,HIT
4,2025-07-22,1.60,2.25,1.42,1.69,1.69,15440600.0,15440600.0,0.09,5.62,1.74,"July 22, 25",0.0562,HIT


In [ ]:
lazy_df = (
    pl.scan_parquet(prices_path)
    .select(pl.col("symbol").unique())
)

In [ ]:
unique_symbols = lazy_df.collect().to_pandas()

In [ ]:
unique_symbols

,symbol
0,GAINZ
1,CCOI
2,BULL
3,AVR
4,FRHC
...,...
1995,CDZI
1996,ALRM
1997,ATLN
1998,FTAIN


In [ ]:
missing_symbols = list(set(symbols_to_get) - set(unique_symbols['symbol'].tolist()))

In [ ]:
pd.DataFrame(missing_symbols, columns=['symbol'])

,symbol
0,QNRX
1,BXRX
2,BIG
3,ELTK
4,OCEAW
...,...
9631,QBTS-WT
9632,MGM
9633,IONQ
9634,RACE


In [ ]:
lazy_df = (
    pl.scan_parquet(prices_path)
    .filter(pl.col("symbol") == "AAPL")
)

In [ ]:
df = lazy_df.collect().to_pandas()

In [ ]:
df

,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,symbol
0,2025-07-28,214.03000,214.85000,213.06000,214.05000,214.05000,36504158.0,36504158.0,0.020000,0.009344,213.99750,"July 28, 25",0.000093,AAPL
1,2025-07-25,214.70000,215.24000,213.40000,213.88000,213.88000,40268781.0,40268781.0,-0.820000,-0.381930,214.30500,"July 25, 25",-0.003819,AAPL
2,2025-07-24,213.90000,215.69000,213.53000,213.76000,213.76000,46022620.0,46022620.0,-0.140000,-0.065451,214.22000,"July 24, 25",-0.000655,AAPL
3,2025-07-23,215.00000,215.15000,212.41000,214.15000,214.15000,46989301.0,46989301.0,-0.850000,-0.395350,214.17750,"July 23, 25",-0.003954,AAPL
4,2025-07-22,213.14000,214.95000,212.23000,214.40000,214.40000,46404100.0,46404100.0,1.260000,0.591160,213.68000,"July 22, 25",0.005912,AAPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6425,2000-01-07,0.86161,0.90179,0.85268,0.88839,0.74760,460734748.0,460734748.0,0.026786,3.110000,0.87612,"January 07, 00",0.031100,AAPL
6426,2000-01-06,0.94754,0.95536,0.84821,0.84821,0.71379,767984764.0,767984764.0,-0.099331,-10.480000,0.89983,"January 06, 00",-0.104800,AAPL
6427,2000-01-05,0.92634,0.98716,0.91964,0.92857,0.78141,778322488.0,778322488.0,0.002233,0.240730,0.94043,"January 05, 00",0.002407,AAPL
6428,2000-01-04,0.96652,0.98772,0.90346,0.91518,0.77014,512377600.0,512377600.0,-0.051339,-5.310000,0.94322,"January 04, 00",-0.053100,AAPL


In [ ]:
# * Step 2: Get the stock prices for all tickers

# * FMP Prices API

# FMP API Notes: 
#  1. This will take 10 minutes to run as it fetches historical prices for 12,000+ assets and convert to a 3 GB parquet file.
#  2. You can adjust the `api_calls_per_minute` parameter based on your FMP plan to avoid hitting rate limits. My limit is 3000 calls per minute, so I set it to 2000 here because I saw I was getting ERRORS when I set it to 3000.

prices_path = client.historical_prices(
    symbols=symbols_to_get[:2000],
    start_date="2000-01-01",
    end_date=TODAY,
    api_calls_per_minute=2000, # Adjust based on your FMP plan
    cache=True,  # This will cache the file in the FMP cache directory
    return_path=True,  # This will return the path to the cached file instead of the DataFrame (which is useful for large datasets)
)

# Inspect the cached file using Polars Lazy Evaluation for large datasets

# * Your cached file path will be different, so you need to adjust this path accordingly

prices_path =  "/Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/" + "data/qsconnect-demo/cache/fmp/historical-prices-2025-07-28.parquet"

print(prices_path)

# Detect Missing Symbols

# Get the head of the DataFrame to see the first few rows
lazy_df = pl.scan_parquet(prices_path)

lazy_df.head

lazy_df.head().collect().to_pandas()

# Get the unique symbols in the DataFrame
lazy_df = (
    pl.scan_parquet(prices_path)
    .select(pl.col("symbol").unique())
)   

unique_symbols = lazy_df.collect().to_pandas()

unique_symbols

# Difference between symbols_to_get and unique_symbols
missing_symbols = list(set(symbols_to_get) - set(unique_symbols['symbol'].tolist()))

pd.DataFrame(missing_symbols, columns=['symbol'])


# Locate a single stock ticker, e.g., Apple (AAPL)

lazy_df = (
    pl.scan_parquet(prices_path)
    .filter(pl.col("symbol") == "AAPL")
)

df = lazy_df.collect().to_pandas()

df
import pytimetk as tk
df.glimpse()

df \
    .groupby('symbol') \
    .plot_timeseries(
        date_column='date',
        value_column='close',
        title='Stock Price',
    )


# * Yahoo Finance API Downloads

# Note: This is a long running operation (about 10 minutes)

client.historical_prices_yahoo(
    symbols=symbols_to_get[:2000],  
    start_date="2000-01-01",
    end_date=TODAY,
    batch_size=100,
)


# * Step 3: Get the fundamental data for all tickers

# - WARNING: Long running operation (134 minutes)
# - Note: Must wait 10 seconds between bulk API requests

client.fetch_bulk_financial_statements(
    statement_type='all', # all available statements: income, balance, cash flow, ...
    periods='all', # annual and quarter 
    
    # start_year=2000,
    start_year=2024,  
    
    end_year=pd.to_datetime(TODAY).year,
    api_buffer_seconds=12,  # Adjust based on your FMP plan to avoid hitting rate limits
)

# * Step 4: Detect Cached Files

client.detect_cached_files()

client.summarize_cached_files()


# Detect Missing Cached Files

df = client.detect_missing_cached_files(
    statement_type="all",
    periods='all',
    start_year=2001,
    end_year=2025, # Try with 2025
    override_date=TODAY,
)

# Clean up Cached Files
help(client.delete_cached_files)

# client.delete_cached_files(df.iloc[[0]])

# * 2.0 BUILD YOUR QUANTAMENTAL FINANCIAL DATABASE ----


# * Step 1: Identify the files you want to include in your database

all_cached_files = client.detect_cached_files(
    # start_date=TODAY,
    # end_date=pd.to_datetime(TODAY) + pd.DateOffset(days=1),  
)

all_cached_files[all_cached_files['file_name'].str.startswith('historical-prices')]

all_cached_files[all_cached_files['file_name'].str.contains('2006')]

# * Step 2: Create a database from the cached files

# This takes a 5 minutes to run as it processes all the cached files and creates the qsconnect DuckDB database

errors_df = client.load_cached_files_to_database(
    cached_files_df=all_cached_files, 
    fresh=True, # Set to True to create a fresh database; set to False to append to an existing database
)

errors_df # If this is empty, then all files were loaded successfully; otherwise a DataFrame with errors will be returned

# * Step 3: Inspect the database

# Connect to the database
client.connect_to_database()

# List All Database Tables
client.list_database_tables()

# Glimpse All Database Tables
client.glimpse_all_database_tables(max_colwidth=25)

# Missing Values Report
client.missing_values_all_database_tables(max_colwidth=25)

# Missing Calendar Years Report for FMP Fundamental Data
client.missing_calendar_years_all_database_tables(
    start_year=2000, 
    end_year=2025,
)

# Quality Report
client.database_quality_report(
    start_year=2000, 
    end_year=2025
)

# Run SQL Queries on the Database
conn = client.connect_to_database()

conn.execute("""
    SELECT *
    FROM historical_prices_fmp
    LIMIT 5
""").fetchdf()

# Disconnect from the database
client.close_database_connection()

# DBCode Extension Demo
# - Use the DBCode extension to examine the database tables


# * 3.0 BUILD YOUR ZIPLINE BUNDLE ----

# * Step 1: Create a Zipline Bundle from the FMP Database

# Connect to the database
client.connect_to_database()

# Create a Zipline Bundle from the database
client.ingest_zipline_bundle_from_fmp_tables(bundle_name="qspro_demo_historical_prices_fmp")

# Disconnect from the database
client.close_database_connection()

# * Step 2: Open the Zipline hidden folder

# Mac users can find the hidden folder at:
#   1. Open Finder.
#   2. Press `Command + Shift + .` (period) to toggle hidden files visibility.
#   3. Navigate to the `~/.zipline` directory.

# Windows users can find the hidden folder at:
#   C:\Users\<YourUsername>\.zipline

2025-07-28 19:57:27,060 - QSConnect - INFO - Loaded cached data for the given date range from cache.


/Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/data/qsconnect-demo/cache/fmp/historical-prices-2025-07-28.parquet
<class 'pandas.core.frame.DataFrame'>: 6430 rows of 14 columns
date:              datetime64[ns]    [Timestamp('2025-07-28 00:00:00'),  ...
open:              float64           [214.03, 214.7, 213.9, 215.0, 213.1 ...
high:              float64           [214.85, 215.24, 215.69, 215.15, 21 ...
low:               float64           [213.06, 213.4, 213.53, 212.41, 212 ...
close:             float64           [214.05, 213.88, 213.76, 214.15, 21 ...
adjClose:          float64           [214.05, 213.88, 213.76, 214.15, 21 ...
volume:            float64           [36504158.0, 40268781.0, 46022620.0 ...
unadjustedVolume:  float64           [36504158.0, 40268781.0, 46022620.0 ...
change:            float64           [0.02, -0.82, -0.14, -0.85, 1.26, 0 ...
changePercent:     float64           [0.00934448, -0.38193, -0.06545115, ...
vwap:              float64           [

Fetching data batches:  30%|███       | 6/20 [02:33<05:56, 25.44s/batch]ERROR [2025-07-28T20:00:33-0400][yfinance] 
1 Failed download:
ERROR [2025-07-28T20:00:33-0400][yfinance] ['CORZW']: YFPricesMissingError('possibly delisted; no price data found  (1d 2000-01-01 -> 2025-07-28)')
Fetching data batches:  65%|██████▌   | 13/20 [05:29<02:51, 24.54s/batch]ERROR [2025-07-28T20:03:30-0400][yfinance] 
1 Failed download:
ERROR [2025-07-28T20:03:30-0400][yfinance] ['BGLWW']: YFPricesMissingError('possibly delisted; no price data found  (1d 2000-01-01 -> 2025-07-28)')
Fetching data batches:  80%|████████  | 16/20 [06:45<01:39, 24.98s/batch]ERROR [2025-07-28T20:04:44-0400][yfinance] 
1 Failed download:
ERROR [2025-07-28T20:04:44-0400][yfinance] ['BKHAR']: YFPricesMissingError('possibly delisted; no price data found  (1d 2000-01-01 -> 2025-07-28)')
Fetching data batches:  90%|█████████ | 18/20 [07:35<00:49, 24.97s/batch]ERROR [2025-07-28T20:05:39-0400][yfinance] 
3 Failed downloads:
ERROR [2025-

Help on method delete_cached_files in module qsconnect.client:

delete_cached_files(cached_files_df: pandas.core.frame.DataFrame) method of qsconnect.client.Client instance
    Delete cached files from the cache directory.
    
    Parameters
    ----------
    cached_files_df : pd.DataFrame
        DataFrame containing the cached files metadata (from `detect_cached_files` function).



Loading files (Batch 1):  30%|███       | 3/10 [00:00<00:01,  5.28it/s]2025-07-28 20:13:07,049 - QSConnect - INFO - Data from bulk-cash-flow-statement-growth_2024_annual-2025-07-28.parquet inserted into table bulk_cash_flow_statement_growth_annual_fmp
2025-07-28 20:13:07,195 - QSConnect - INFO - Data from bulk-key-metrics_2024_annual-2025-07-28.parquet inserted into table bulk_key_metrics_annual_fmp
Loading files (Batch 1): 100%|██████████| 10/10 [00:02<00:00,  3.87it/s]
2025-07-28 20:13:08,915 - QSConnect - INFO - Processing batch 11-20 of 21
Loading files (Batch 2): 100%|██████████| 10/10 [00:02<00:00,  4.21it/s]
2025-07-28 20:13:11,291 - QSConnect - INFO - Processing batch 21-21 of 21
Loading files (Batch 3): 100%|██████████| 1/1 [00:00<00:00,  2.31it/s]
2025-07-28 20:13:11,725 - QSConnect - INFO - Converting date column types...
2025-07-28 20:13:11,737 - QSConnect - INFO - Converting column 'date' to 'TIMESTAMP_NS' in all tables...
Converting columns: 100%|██████████| 21/21 [00:00<



Table 1: 'bulk_balance_sheet_statement_annual_fmp'
Rows: 49660
Columns: 53
--------------------------
                     name         type        sample value
                   symbol      VARCHAR           000001.SZ
                     date TIMESTAMP_NS 2024-12-31 00:00:00
         reportedCurrency      VARCHAR                 CNY
                      cik       DOUBLE                 0.0
              fillingDate TIMESTAMP_NS 2024-12-31 00:00:00
             acceptedDate TIMESTAMP_NS 2024-12-31 00:00:00
             calendarYear       BIGINT                2024
                   period      VARCHAR                  FY
   cashAndCashEquivalents       DOUBLE      680935000000.0
     shortTermInvestments       DOUBLE                 0.0
cashAndShortTermInvest...       DOUBLE      680935000000.0
           netReceivables       DOUBLE                 0.0
                inventory       DOUBLE                 0.0
       otherCurrentAssets       DOUBLE                 0.0
       tota

2025-07-28 20:13:13,286 - QSConnect - INFO - Connected to DuckDB at /Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/data/qsconnect-demo/database/qsconnect.duckdb


2025-07-28 20:13:13,374 - QSConnect - INFO - Connection to DuckDB Database closed successfully: /Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/data/qsconnect-demo/database/qsconnect.duckdb
2025-07-28 20:13:13,383 - QSConnect - INFO - Connected to DuckDB at /Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/data/qsconnect-demo/database/qsconnect.duckdb
INFO [2025-07-28T20:13:25-0400][zipline.data.bundles.core] Ingesting qspro_demo_historical_prices_fmp
2025-07-28 20:13:25,040 - QSConnect - INFO - Initializing Data Loader.
2025-07-28 20:13:25,041 - QSConnect - INFO - Fetching data from database.
2025-07-28 20:13:25,048 - QSConnect - INFO - Processing 2000 unique symbols
2025-07-28 20:13:25,049 - QSConnect - INFO - Table schema validation passed.
2025-07-28 20:13:25,236 - QSConnect - INFO - Validating data quality.
2025-07-28 20:13:25,347 - QSConnect - INFO - Data validation complete.
2025-07-28 20:13:25,355 - QSConnect - INFO - Generating asset metadata.
2025-07

No kernel connected

No kernel connected

No kernel connected